In [ ]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Input, BatchNormalization, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K

from keras.models import load_model


import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 110
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer

In [ ]:
# Set the path to the base directory of frequency-domain hdf files
base_dir = ''

In [ ]:
loads = ['05', '20', '25', '40']
reps = ['0', '1']
states = ['r1b', 'r2b', 'r3b', 'r4b', 'rs']

In [ ]:
def addresses_generator(signal, loads, states, reps):
    results = {}
    for load in loads:
        results[load] = []
        for state in states:
            results[load].append([signal + '_' + state +'_torque' + load + '_' + rep for rep in reps])

    return results

In [ ]:
Vib_acpe_addresses = addresses_generator('Vib_acpe', loads, states, reps)
Vib_acpi_addresses = addresses_generator('Vib_acpi', loads, states, reps)
Vib_axial_addresses = addresses_generator('Vib_axial', loads, states, reps)
Vib_base_addresses = addresses_generator('Vib_base', loads, states, reps)
Vib_carc_addresses = addresses_generator('Vib_carc', loads, states, reps)

In [ ]:
def df_loader_from_addresses(addresses, base_dir, signal):
    results = {}
    for load in list(addresses.keys()):
        results[load] = []
        for addresses_of_same_load_state_reps in addresses[load]:
            list_of_dfs = [pd.read_hdf(base_dir + signal+ '//' + address + '.hdf', address) for address in addresses_of_same_load_state_reps]
            concat_dfs = pd.concat(list_of_dfs)
            concat_dfs = concat_dfs.reset_index(drop=True)
            results[load].append(concat_dfs)
        results[load] = pd.concat(results[load])
        results[load] = results[load].reset_index(drop=True)
    return results

In [ ]:
Vib_acpe_dict = df_loader_from_addresses(Vib_acpe_addresses, base_dir, 'Vib_acpe', )
Vib_acpi_dict = df_loader_from_addresses(Vib_acpi_addresses, base_dir, 'Vib_acpi', )
Vib_axial_dict = df_loader_from_addresses(Vib_axial_addresses, base_dir, 'Vib_axial', )
Vib_base_dict = df_loader_from_addresses(Vib_base_addresses, base_dir, 'Vib_base')
Vib_carc_dict = df_loader_from_addresses(Vib_carc_addresses, base_dir, 'Vib_carc',)

In [ ]:
def train_test_splitter(signal_dict):
  signal_train = {}
  signal_test = {}

  for load in list(signal_dict.keys()):
    temp_train, temp_test = train_test_split(signal_dict[load], test_size=0.25, random_state=42)
    
    signal_train[load] = temp_train
    signal_test[load] = temp_test

  return signal_train, signal_test

In [ ]:
Vib_acpe_train, Vib_acpe_test = train_test_splitter(Vib_acpe_dict)
Vib_acpi_train, Vib_acpi_test = train_test_splitter(Vib_acpi_dict)
Vib_axial_train, Vib_axial_test = train_test_splitter(Vib_axial_dict)
Vib_base_train, Vib_base_test = train_test_splitter(Vib_base_dict)
Vib_carc_train, Vib_carc_test = train_test_splitter(Vib_carc_dict)

In [ ]:
Vib_acpe_train['total'] = np.concatenate([Vib_acpe_train[load] for load in list(Vib_acpe_train.keys())])
Vib_acpe_test['total'] = np.concatenate([Vib_acpe_test[load] for load in list(Vib_acpe_test.keys())])

Vib_acpi_train['total'] = np.concatenate([Vib_acpi_train[load] for load in list(Vib_acpi_train.keys())])
Vib_acpi_test['total'] = np.concatenate([Vib_acpi_test[load] for load in list(Vib_acpi_test.keys())])

Vib_axial_train['total'] = np.concatenate([Vib_axial_train[load] for load in list(Vib_axial_train.keys())])
Vib_axial_test['total'] = np.concatenate([Vib_axial_test[load] for load in list(Vib_axial_test.keys())])

Vib_base_train['total'] = np.concatenate([Vib_base_train[load] for load in list(Vib_base_train.keys())])
Vib_base_test['total'] = np.concatenate([Vib_base_test[load] for load in list(Vib_base_test.keys())])

Vib_carc_train['total'] = np.concatenate([Vib_carc_train[load] for load in list(Vib_carc_train.keys())])
Vib_carc_test['total'] = np.concatenate([Vib_carc_test[load] for load in list(Vib_carc_test.keys())])


In [ ]:
def train_test_scaler(signal_train, signal_test):

  for load in list(signal_train.keys()):
    temp_scaler = MinMaxScaler()
    temp_scaler.fit(signal_train[load])
    temp_train_scaled = temp_scaler.transform(signal_train[load])
    temp_test_scaled = temp_scaler.transform(signal_test[load])

    signal_train[load] = temp_train_scaled
    signal_test[load] = temp_test_scaled

  return signal_train, signal_test

In [ ]:
Vib_acpe_train_scaled, Vib_acpe_test_scaled = train_test_scaler(Vib_acpe_train, Vib_acpe_test)
Vib_acpi_train_scaled, Vib_acpi_test_scaled = train_test_scaler(Vib_acpi_train, Vib_acpi_test)
Vib_axial_train_scaled, Vib_axial_test_scaled = train_test_scaler(Vib_axial_train, Vib_axial_test)
Vib_base_train_scaled, Vib_base_test_scaled = train_test_scaler(Vib_base_train, Vib_base_test)
Vib_carc_train_scaled, Vib_carc_test_scaled = train_test_scaler(Vib_carc_train, Vib_carc_test)

In [ ]:
labels = np.concatenate([
    np.full((300), 1),
    np.full((300), 2),
    np.full((300), 3),
    np.full((300), 4),
    np.full((300), 0),
])
labels.shape
labels_df = pd.DataFrame({'target': labels})

In [ ]:
lables_bin = LabelBinarizer().fit_transform(y=labels_df['target'])
labels_bin_train, labels_bin_test = train_test_split(lables_bin, test_size=0.25, random_state=42)

In [ ]:
labels_bin_train_dict = dict(zip(loads, [labels_bin_train, labels_bin_train, labels_bin_train, labels_bin_train]))
labels_bin_train_dict['total'] = np.concatenate([labels_bin_train, labels_bin_train, labels_bin_train, labels_bin_train])

In [ ]:
labels_bin_test_dict = dict(zip(loads,[labels_bin_test, labels_bin_test, labels_bin_test, labels_bin_test]))
labels_bin_test_dict['total'] = np.concatenate([labels_bin_test, labels_bin_test, labels_bin_test, labels_bin_test])

In [ ]:
def vibration_concatanator(Vib_acpe_train_scaled, Vib_acpi_train_scaled, Vib_axial_train_scaled, Vib_base_train_scaled, Vib_carc_train_scaled):
  concatanated_dict = {}
  for load in list(Vib_acpe_train_scaled.keys()):
    concatanated_dict[load] = np.concatenate((Vib_acpe_train_scaled[load],
                                              Vib_acpi_train_scaled[load],
                                              Vib_axial_train_scaled[load],
                                              Vib_base_train_scaled[load],
                                              Vib_carc_train_scaled[load]), axis=1)

  return concatanated_dict

In [ ]:
Vib_train_scaled = vibration_concatanator(Vib_acpe_train_scaled, Vib_acpi_train_scaled, Vib_axial_train_scaled, Vib_base_train_scaled, Vib_carc_train_scaled)

In [ ]:
Vib_test_scaled = vibration_concatanator(Vib_acpe_test_scaled, Vib_acpi_test_scaled, Vib_axial_test_scaled, Vib_base_test_scaled, Vib_carc_test_scaled)

In [ ]:
for load in list(Vib_train_scaled.keys()):
  print(load, '   ', Vib_train_scaled[load].shape, labels_bin_train_dict[load].shape)

In [ ]:
for load in list(Vib_acpe_test.keys()):
  print(load, '   ', Vib_test_scaled[load].shape, labels_bin_test_dict[load].shape)

In [ ]:
def model_creator():

  input = Input(shape=2560, name='input1')
  input_emb = Dense(units=1280, activation='tanh', name = 'HL1_1')(input)
  input_emb = Dense(units=640, activation='tanh', name = 'HL1_2')(input_emb)
  input_emb = Dense(units=580, activation='tanh', name = 'HL1_3')(input_emb)
  input_emb = Dense(units=512, activation='tanh', name = 'HL1_4')(input_emb)
  input_emb = Dense(units=256, activation='tanh', name = 'HL1_5')(input_emb)
  input_emb = Dense(units=128, activation='tanh', name = 'HL1_6')(input_emb)
  input_emb = Dense(units=64, activation='tanh', name = 'HL1_7')(input_emb)
  predicted_label = Dense(units=5, activation='softmax', name='label')(input_emb)

  return Model(inputs = input, outputs = predicted_label)

In [ ]:
model = model_creator()
plot_model(model, show_shapes=True)

In [ ]:
class SaveBestModel(tf.keras.callbacks.Callback):
    def __init__(self, save_best_metric='val_loss', this_max=False):
        self.save_best_metric = save_best_metric
        self.max = this_max
        if this_max:
            self.best = float('-inf')
        else:
            self.best = float('inf')

    def on_epoch_end(self, epoch, logs=None):
        metric_value = logs[self.save_best_metric]
        if self.max:
            if metric_value > self.best:
                self.best = metric_value
                self.best_weights = self.model.get_weights()

        else:
            if metric_value < self.best:
                self.best = metric_value
                self.best_weights= self.model.get_weights()

In [ ]:
lr = 0.000001
ep = 400
n_repetition = 1

training_classification_accs = []
testing_classification_accs = []
histories = {}

In [ ]:
for i in range(n_repetition):
  temp_training_accs = []
  temp_testing_accs = []
  for load in list(Vib_acpe_train.keys()):

    model = model_creator()
    best_model = SaveBestModel()
    opt = optimizers.Adam(learning_rate=lr, decay=lr / ep)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    temp_histories  = model.fit(Vib_train_scaled[load], labels_bin_train_dict[load],
                                validation_split=0.25, epochs = ep, callbacks=[best_model])
    
    model.set_weights(best_model.best_weights)
    
    histories[load + '_' + str(i)] = temp_histories
    
    temp_temp_train = []
    temp_temp_test = []
    
    for evaluation_load in list(Vib_acpe_train.keys()):
      temp_temp_train.append(model.evaluate(Vib_train_scaled[evaluation_load], labels_bin_train_dict[evaluation_load],)[1])
      
      temp_temp_test.append(model.evaluate(Vib_test_scaled[evaluation_load], labels_bin_test_dict[evaluation_load],)[1])

    temp_training_accs.append(temp_temp_train)
    temp_testing_accs.append(temp_temp_test)



  training_classification_accs.append(temp_training_accs)
  testing_classification_accs.append(temp_testing_accs)

In [ ]:
for item in training_classification_accs:
  print(pd.DataFrame(item, columns=list(Vib_acpe_train.keys()), index=list(Vib_acpe_train.keys())), '\n')

In [ ]:
for item in testing_classification_accs:
  print(pd.DataFrame(item, columns=list(Vib_acpe_train.keys()), index=list(Vib_acpe_train.keys())), '\n')

In [ ]:
for item in list(histories.keys()):
  plt.plot(histories[item].history['loss'])
  plt.plot(histories[item].history['val_loss'])
  plt.title('model loss - {}'.format(item))
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()

In [ ]:
for item in list(histories.keys()):
  plt.plot(histories[item].history['accuracy'])
  plt.plot(histories[item].history['val_accuracy'])
  plt.title('model accuracy - {}'.format(item))
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()